In [11]:
import os
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline


import glob
from astropy.io import fits
from astropy.stats import sigma_clip
from scipy.ndimage import interpolation as interp

#from skimage.feature.register_translation import (register_translation, _upsampled_dft)

# import aotools
import astroscrappy
from scipy.optimize import leastsq
# import ccdproc
import math

In [12]:
from scipy.ndimage import median_filter, gaussian_filter


def guess_gaussian_parameters(d):
    """
    This image guesses the maximum intensity of the 
    image by obtaining a smoothed version of the image 
    via median + gaussian filtering. Then, finds the 
    maximum of the smoothed image. Using the median-filtered 
    image, the algorithm also estimates the width of the PSF and 
    with this value and an estimate of the volume, the amplitude of 
    such gaussian.

    Input

    d       Numpy array that contains the values of the pixels.

    Output

    x0      x coordinate of the image maximum
    
    y0      y coordinate of the image maximum

    sigma   Width of the PSF

    A       Estimated amplitude of the gaussian function
    """

    # First, smooth the image with a median filter. For this, find 
    # the optimal window as the square-root of the geometric mean 
    # between the sizes of the array. This step is good to kill outliers:
    window = int(np.sqrt(np.sqrt(d.shape[0] * d.shape[1])))
    if window % 2 == 0:
        window = window + 1

        # Originally there was a median filter here, but it really adds a lot of time to the routine
    #d_mfiltered = median_filter(d,size = window)
    # Going to try without it to see if it makes a significant difference for vetoing
    d_gfiltered = gaussian_filter(d, sigma=window)

    # Now, find the maximum of this image:
    y0, x0 = np.where(d_gfiltered == np.max(d_gfiltered))

    # Take the first element. This helps in two cases: (1) only one maximum has 
    # been found, the outputs are numpy arrays and you want to extract the numbers 
    # and (2) in case there are several maximums (this has not happened so 
    # far but could in principle), pick the first of them:
    y0 = y0[0]
    x0 = x0[0]

    # Now estimate the width of the PSF by taking a "cross-plot" using the 
    # maximum values found:
    x_cut = d[:, int(x0)]
    sigma_x = (np.sum(x_cut * (np.abs(np.arange(len(x_cut)) - y0))) / np.sum(x_cut)) / 3.
    y_cut = d[int(y0), :]
    sigma_y = (np.sum(y_cut * (np.abs(np.arange(len(y_cut)) - x0))) / np.sum(y_cut)) / 3.
    sigma = np.sqrt(sigma_x * sigma_y)

    # (Under) estimate amplitude assuming a gaussian function:
    A = (np.sum(d - np.median(d)) / (2. * np.pi * sigma ** 2))

    return x0, y0, sigma, 2. * A


In [13]:
datadir = '/home/tehan/PycharmProjects/Shane-AO-Reduction/Raw_Data/data-2024-01-28-AO-Paul.Robertson/'
os.chdir(datadir)

#These are the id numbers of the fitz files
firstID = 1;
lastID = 9999;

##Let's go through our images and remove the bad frames

all_object_list = []

veto_list = []

#Let's just make a list of every object
for ID in range(firstID, lastID + 1):
    try:
        if ID < 100:
            if ('s00' + str(ID) + '.fits'):
                header = fits.getheader('s00' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s00' + str(ID) + '.fits')


        elif 99 < ID < 1000:
            if ('s0' + str(ID) + '.fits'):
                header = fits.getheader('s0' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s0' + str(ID) + '.fits')

        elif 999 < ID < 10000:
            if ('s' + str(ID) + '.fits'):
                header = fits.getheader('s' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s' + str(ID) + '.fits')

    except OSError:
        continue
    except KeyError:
        continue

flat_obj_list = []
#let's remove the darks
for object in all_object_list:
    header = fits.getheader(object)
    obj = header['OBJECT']
    if obj == 'dark':
        continue
    else:
        flat_obj_list.append(object)




In [14]:
#Now we go through each object here, and we make a list of all the bad ones

veto_list = []

for obj in flat_obj_list:
    header = fits.getheader(obj)
    try:
        image_data = fits.getdata(obj)
        center=(1100,730)
        image_window = image_data[(center[1] - 200):(center[1] + 200), (center[0] - 200):(center[0] + 200)]  # we use the window we care about
        # shrink the window to avoid companion

        print(obj)
        print(header['OBJECT'])
        if 'flat' in header['OBJECT'].lower():
            cent_counts = np.mean(image_window)
            print('Flat!')
        else:

            x0, y0, sigma, A = guess_gaussian_parameters(
                image_window)  #need to do this for the window, not the whole image

            print(image_window[y0,x0])
            #now that we have the centroid, let's make a window about the centroid, and take the average
            centroid_window = image_window[y0 - 1:y0 + 1, x0 - 1:x0 + 1]
            cent_counts = np.mean(centroid_window)  #let's try max instead of mean

        print(cent_counts)

        if cent_counts > 25000:  #Setting our max counts at 25000 here
            veto_list.append(obj)
            print('Appended!')
        elif cent_counts < 500:  #Setting our minimum counts at 6500 here
            veto_list.append(obj)
            print('Appended!')
        elif math.isnan(cent_counts) == True:
            veto_list.append(obj)
            print('Appended!')
        else:
            continue
    except ValueError:
        continue


#Just median filter: ~94 s, appears accurate
#Just Gaussian filter: ~ 1 s, appears accurate



s0010.fits
dome flat
Flat!
13205.076
s0011.fits
dome flat
Flat!
13206.251
s0012.fits
dome flat
Flat!
13190.416
s0013.fits
dome flat
Flat!
13219.862
s0014.fits
dome flat
Flat!
13214.646
s0015.fits
dome flat
Flat!
13218.309
s0016.fits
dome flat
Flat!
13206.309
s0017.fits
dome flat
Flat!
13199.102
s0018.fits
dome flat
Flat!
13201.97
s0019.fits
dome flat
Flat!
13213.73
s0020.fits
dome flat
Flat!
11351.138
s0021.fits
dome flat
Flat!
11265.767
s0022.fits
dome flat
Flat!
11264.504
s0023.fits
dome flat
Flat!
11264.052
s0024.fits
dome flat
Flat!
11263.176
s0025.fits
dome flat
Flat!
11272.488
s0026.fits
dome flat
Flat!
11270.747
s0027.fits
dome flat
Flat!
11267.581
s0028.fits
dome flat
Flat!
11272.258
s0029.fits
dome flat
Flat!
11272.493
s0030.fits
dome flat
Flat!
11272.966
s0031.fits
dome flat
Flat!
11495.198
s0032.fits
dome flat
Flat!
11489.968
s0033.fits
dome flat
Flat!
11496.177
s0034.fits
dome flat
Flat!
11490.955
s0035.fits
dome flat
Flat!
11496.264
s0036.fits
dome flat
Flat!
11488.317
s00

/tmp/ipykernel_7652/1239570007.py:57: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(sigma_x * sigma_y)


11230.5625
11663.906
s0437.fits
TIC_283866910
14915.125
14230.4375
s0438.fits
TIC_283866910
21778.875
20576.469
s0439.fits
TIC_283866910
17490.375
16856.812
s0440.fits
TIC_283866910
18995.0
18079.312
s0441.fits
TIC_283866910
19885.125
18725.281
s0442.fits
TIC_283866910
21199.125
19792.188
s0443.fits
TIC_283866910
18835.0
17792.844
s0444.fits
TIC_283866910
18474.188
17485.438
s0445.fits
TIC_283866910
14763.0625
13742.094
s0446.fits
TIC_283866910
953.375
922.09375
s0447.fits
TIC_283866910
935.75
919.71875
s0448.fits
TIC_283866910
889.5
876.15625
s0449.fits
TIC_283866910
867.75
865.53125
s0450.fits
TIC_283866910
924.4375
882.59375
s0451.fits
TIC_283866910
922.75
886.90625
s0452.fits
TIC_283866910
882.1875
849.03125
s0453.fits
TIC_283866910
888.375
873.40625
s0454.fits
TIC_283866910
895.25
888.6875
s0455.fits
TIC_283866910
888.875
863.15625
s0456.fits
TIC_283866910
10285.0625
9784.719
s0457.fits
TIC_283866910
11381.1875
10619.219
s0458.fits
TIC_283866910
10572.625
10252.094
s0459.fits
TIC_

In [15]:
#Write it to a text file for later
with open('veto_list.txt', 'w') as f:
    for item in veto_list:
        f.write("%s\n" % item)